## Prepare environment

Install libraries

In [ ]:
%%capture
!pip install vllm
!pip install Jinja2==3.1.3
!pip install transformers==4.37.2
!pip install bitsandbytes==0.42.0
!pip install accelerate==0.26.1
!pip install flash-attn --no-build-isolation --upgrade
!pip install -U datasets
!pip install peft==0.3.0
!pip install git+https://github.com/huggingface/peft
!pip install evaluate
!pip install rouge_score

Import libraries

In [ ]:
import torch
from peft import (
    PeftModel,
    PeftConfig,
    get_peft_model,
    PromptTuningInit,
    PromptTuningConfig,
    PrefixTuningConfig,
    LoraConfig,
    TaskType,
    PeftType,
    PeftConfig,
    PeftModel,
    prepare_model_for_kbit_training,
    get_peft_model,
    LoraConfig
)
# prepare_model_for_int8_training
import time
import evaluate
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yaml
from datasets import Dataset
from transformers import AutoConfig
# from peft import prepare_model_for_int
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoModelForSeq2SeqLM,
    AutoConfig,
    TrainingArguments,
    Trainer,
    default_data_collator,
    DataCollatorForLanguageModeling
)
from peft import prepare_model_for_kbit_training

## Read data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dataset = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/data/table_insights_labeled_data.xlsx', sheet_name="Data")

In [ ]:
dataset

,DeviceMaskedID,data (train/ test),date,cpu_usage,ram_usage,diskio_usage
0,Device_1,Train,2022-08-02,7.845169,52.662081,NaN
1,Device_1,Train,2022-08-03,5.029416,53.519746,NaN
2,Device_1,Train,2022-08-04,4.855019,53.656122,NaN
3,Device_1,Train,2022-08-05,8.014844,55.245124,NaN
4,Device_1,Train,2022-08-07,16.909919,54.216115,NaN
...,...,...,...,...,...,...
5276,Device_215,Train,2022-05-25,NaN,NaN,90.005963
5277,Device_215,Train,2022-05-26,NaN,NaN,75.829685
5278,Device_215,Train,2022-05-27,NaN,NaN,88.540792
5279,Device_215,Train,2022-05-28,NaN,NaN,77.175451


In [ ]:

labels = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/data/table_insights_labeled_data.xlsx', sheet_name="Labels")

In [ ]:
labels

,DeviceMaskedID,data (train/ test),insight_0,insight_1,insight_2
0,Device_1,Train,The data contains information about a computer...,1. The highest CPU usage was recorded on 222-8...,1. The diskio_usage column has no data (NaN) f...
1,Device_2,Train,The data contains CPU usage information for a ...,1. CPU usage has been consistently increasing ...,"Based on the provided data, the top 3 aberrati..."
2,Device_3,Train,The data contains information about a user's c...,1. The highest CPU usage was recorded on 223-3...,1. High CPU usage on 223-3-13: The CPU usage o...
3,Device_4,Test,The data contains information about a user's c...,1. High CPU usage days: The user experienced t...,1. Missing Data: The 'diskio_usage' column has...
4,Device_5,Train,The data contains information about a user's c...,1. High CPU Usage: The highest CPU usage recor...,1. Missing Data: The 'diskio_usage' column has...
...,...,...,...,...,...
210,Device_211,Train,The data contains information about a user's c...,1. High RAM Usage: The RAM usage is consistent...,"1. Highest CPU usage: On 223-3-1, the CPU usag..."
211,Device_212,Train,The data contains information about a user's c...,1. Significant increase in resource usage towa...,1. There is a significant increase in CPU usag...
212,Device_213,Test,The data contains information about a user's c...,1. Highest CPU usage: The highest CPU usage wa...,1. Missing Data: The 'diskio_usage' column has...
213,Device_214,Train,The data contains daily disk IO usage for a co...,1. The highest disk IO usage was recorded on 1...,"After analyzing the dataset, the top 3 aberrat..."


In [ ]:
data_dictionary = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/data/table_insights_labeled_data.xlsx', sheet_name="Data dictionary")

In [ ]:
dataset.columns

Index(['DeviceMaskedID', 'data (train/ test)', 'date', 'cpu_usage',
       'ram_usage', 'diskio_usage'],
      dtype='object')

## Split train and test data

In [ ]:
dataset["data (train/ test)"].unique()

array(['Train', 'Test'], dtype=object)

In [ ]:
dataset["data (train/ test)"].value_counts()

data (train/ test)
Train    4288
Test      993
Name: count, dtype: int64

In [ ]:
dataset.shape

(5281, 6)

In [ ]:
train = dataset.loc[dataset["data (train/ test)"] == "Train"]

In [ ]:
test = dataset.loc[dataset["data (train/ test)"] == "Test"]

In [ ]:
test.columns

Index(['DeviceMaskedID', 'data (train/ test)', 'date', 'cpu_usage',
       'ram_usage', 'diskio_usage'],
      dtype='object')

In [ ]:
print(train.shape)
print(test.shape)

(4288, 6)
(993, 6)


### Prepare Data

In [ ]:
train

,DeviceMaskedID,data (train/ test),date,cpu_usage,ram_usage,diskio_usage
0,Device_1,Train,2022-08-02,7.845169,52.662081,NaN
1,Device_1,Train,2022-08-03,5.029416,53.519746,NaN
2,Device_1,Train,2022-08-04,4.855019,53.656122,NaN
3,Device_1,Train,2022-08-05,8.014844,55.245124,NaN
4,Device_1,Train,2022-08-07,16.909919,54.216115,NaN
...,...,...,...,...,...,...
5276,Device_215,Train,2022-05-25,NaN,NaN,90.005963
5277,Device_215,Train,2022-05-26,NaN,NaN,75.829685
5278,Device_215,Train,2022-05-27,NaN,NaN,88.540792
5279,Device_215,Train,2022-05-28,NaN,NaN,77.175451


In [ ]:
labels

,DeviceMaskedID,data (train/ test),insight_0,insight_1,insight_2
0,Device_1,Train,The data contains information about a computer...,1. The highest CPU usage was recorded on 222-8...,1. The diskio_usage column has no data (NaN) f...
1,Device_2,Train,The data contains CPU usage information for a ...,1. CPU usage has been consistently increasing ...,"Based on the provided data, the top 3 aberrati..."
2,Device_3,Train,The data contains information about a user's c...,1. The highest CPU usage was recorded on 223-3...,1. High CPU usage on 223-3-13: The CPU usage o...
3,Device_4,Test,The data contains information about a user's c...,1. High CPU usage days: The user experienced t...,1. Missing Data: The 'diskio_usage' column has...
4,Device_5,Train,The data contains information about a user's c...,1. High CPU Usage: The highest CPU usage recor...,1. Missing Data: The 'diskio_usage' column has...
...,...,...,...,...,...
210,Device_211,Train,The data contains information about a user's c...,1. High RAM Usage: The RAM usage is consistent...,"1. Highest CPU usage: On 223-3-1, the CPU usag..."
211,Device_212,Train,The data contains information about a user's c...,1. Significant increase in resource usage towa...,1. There is a significant increase in CPU usag...
212,Device_213,Test,The data contains information about a user's c...,1. Highest CPU usage: The highest CPU usage wa...,1. Missing Data: The 'diskio_usage' column has...
213,Device_214,Train,The data contains daily disk IO usage for a co...,1. The highest disk IO usage was recorded on 1...,"After analyzing the dataset, the top 3 aberrat..."


In [ ]:
train

,DeviceMaskedID,data (train/ test),date,cpu_usage,ram_usage,diskio_usage
0,Device_1,Train,2022-08-02,7.845169,52.662081,NaN
1,Device_1,Train,2022-08-03,5.029416,53.519746,NaN
2,Device_1,Train,2022-08-04,4.855019,53.656122,NaN
3,Device_1,Train,2022-08-05,8.014844,55.245124,NaN
4,Device_1,Train,2022-08-07,16.909919,54.216115,NaN
...,...,...,...,...,...,...
5276,Device_215,Train,2022-05-25,NaN,NaN,90.005963
5277,Device_215,Train,2022-05-26,NaN,NaN,75.829685
5278,Device_215,Train,2022-05-27,NaN,NaN,88.540792
5279,Device_215,Train,2022-05-28,NaN,NaN,77.175451


In [ ]:
train_merged = train.merge(labels[["DeviceMaskedID", "insight_2"]], on=["DeviceMaskedID"]).rename(columns={"insight_2": "Summary"})
train_labels = train_merged[["DeviceMaskedID", "Summary"]].drop_duplicates(subset=["DeviceMaskedID"])
train_labels

,DeviceMaskedID,Summary
0,Device_1,1. The diskio_usage column has no data (NaN) f...
24,Device_2,"Based on the provided data, the top 3 aberrati..."
39,Device_3,1. High CPU usage on 223-3-13: The CPU usage o...
54,Device_5,1. Missing Data: The 'diskio_usage' column has...
75,Device_6,1. There is a significant increase in CPU usag...
...,...,...
4183,Device_210,1. Missing Data: The 'diskio_usage' column has...
4206,Device_211,"1. Highest CPU usage: On 223-3-1, the CPU usag..."
4226,Device_212,1. There is a significant increase in CPU usag...
4251,Device_214,"After analyzing the dataset, the top 3 aberrat..."


In [ ]:
train_merged

,DeviceMaskedID,data (train/ test),date,cpu_usage,ram_usage,diskio_usage,Summary
0,Device_1,Train,2022-08-02,7.845169,52.662081,NaN,1. The diskio_usage column has no data (NaN) f...
1,Device_1,Train,2022-08-03,5.029416,53.519746,NaN,1. The diskio_usage column has no data (NaN) f...
2,Device_1,Train,2022-08-04,4.855019,53.656122,NaN,1. The diskio_usage column has no data (NaN) f...
3,Device_1,Train,2022-08-05,8.014844,55.245124,NaN,1. The diskio_usage column has no data (NaN) f...
4,Device_1,Train,2022-08-07,16.909919,54.216115,NaN,1. The diskio_usage column has no data (NaN) f...
...,...,...,...,...,...,...,...
4283,Device_215,Train,2022-05-25,NaN,NaN,90.005963,"Based on the provided data, the top 3 aberrati..."
4284,Device_215,Train,2022-05-26,NaN,NaN,75.829685,"Based on the provided data, the top 3 aberrati..."
4285,Device_215,Train,2022-05-27,NaN,NaN,88.540792,"Based on the provided data, the top 3 aberrati..."
4286,Device_215,Train,2022-05-28,NaN,NaN,77.175451,"Based on the provided data, the top 3 aberrati..."


In [ ]:
train_merged["DeviceMaskedID"].nunique()

172

In [ ]:
train_df = pd.DataFrame(columns=["DeviceMaskedID", "Instruction"])
test_df = pd.DataFrame(columns=["DeviceMaskedID", "Instruction"])

Prepare the train data with instruction

In [ ]:
for device_id in train["DeviceMaskedID"].unique():
  # print(device_id)
  df = train.loc[train["DeviceMaskedID"]==device_id]

  summary = train_labels.loc[train_labels["DeviceMaskedID"] == device_id].reset_index(drop=True).iloc[0, 1]
  df = df.loc[:, ~df.columns.isin(["DeviceMaskedID", "data (train/ test)", "Summary"])]
  df["date"] = df["date"].dt.strftime('%Y-%m-%d')
  df = df.round(2)

  df_str = df.to_json(orient="records", lines=True)

  prompt = f"""Analyze the following table data and provide a creative summary in not more than 3 points:

  {df_str}

  Summary:
  """

  # new_record = {"DeviceMaskedID": device_id, "Instruction": prompt}
  train_df.loc[len(train_df.index)] = [device_id, prompt]
print(train_df.head())
print(train_df.shape)

  DeviceMaskedID                                        Instruction
0       Device_1  Analyze the following table data and provide a...
1       Device_2  Analyze the following table data and provide a...
2       Device_3  Analyze the following table data and provide a...
3       Device_5  Analyze the following table data and provide a...
4       Device_6  Analyze the following table data and provide a...
(172, 2)


<ipython-input-34-6bc9fe65a7a6>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["date"] = df["date"].dt.strftime('%Y-%m-%d')
<ipython-input-34-6bc9fe65a7a6>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["date"] = df["date"].dt.strftime('%Y-%m-%d')
<ipython-input-34-6bc9fe65a7a6>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas

In [ ]:
train_df

,DeviceMaskedID,Instruction
0,Device_1,Analyze the following table data and provide a...
1,Device_2,Analyze the following table data and provide a...
2,Device_3,Analyze the following table data and provide a...
3,Device_5,Analyze the following table data and provide a...
4,Device_6,Analyze the following table data and provide a...
...,...,...
167,Device_210,Analyze the following table data and provide a...
168,Device_211,Analyze the following table data and provide a...
169,Device_212,Analyze the following table data and provide a...
170,Device_214,Analyze the following table data and provide a...


Split validation data from train data

In [ ]:
validation_df = train_df.iloc[1:172:20]
train_df = train_df[~train_df.index.isin(validation_df.index)]

print(validation_df.shape)
print(train_df.shape)

(9, 2)
(163, 2)


In [ ]:
labels.shape

(215, 5)

In [ ]:
# train_merged = train_df.merge(labels[["DeviceMaskedID", "insight_2"]], on=["DeviceMaskedID"])
# # train_merged.set_index("DeviceMaskedID")
# # train_dataset = train_merged[["Instruction"]]
# train_labels = train_merged[["insight_2"]].astype(str).rename(columns={"insight_2": "Summary"})
# train_labels

In [ ]:
train_df

,DeviceMaskedID,Instruction
0,Device_1,Analyze the following table data and provide a...
2,Device_3,Analyze the following table data and provide a...
3,Device_5,Analyze the following table data and provide a...
4,Device_6,Analyze the following table data and provide a...
5,Device_7,Analyze the following table data and provide a...
...,...,...
167,Device_210,Analyze the following table data and provide a...
168,Device_211,Analyze the following table data and provide a...
169,Device_212,Analyze the following table data and provide a...
170,Device_214,Analyze the following table data and provide a...


Prepare test data with Instruction

In [ ]:
for device_id in test["DeviceMaskedID"].unique():
  df = test.loc[test["DeviceMaskedID"]==device_id]
  df = df.loc[:, ~df.columns.isin(["DeviceMaskedID", "data (train/ test)"])]
  df["date"] = df["date"].dt.strftime('%Y-%m-%d')
  df = df.round(2)

  df_str = df.to_json(orient="records", lines=True)

  prompt = f"""Analyze the following table data and provide a creative summary in not more than 3 points:

  {df_str}

  Summary:
  """

  new_record = {"DeviceMaskedID": device_id, "Instruction": prompt}
  test_df.loc[len(test_df.index)] = [device_id, prompt]
print(test_df.head())
print(test_df.shape)

  DeviceMaskedID                                        Instruction
0       Device_4  Analyze the following table data and provide a...
1       Device_8  Analyze the following table data and provide a...
2      Device_16  Analyze the following table data and provide a...
3      Device_17  Analyze the following table data and provide a...
4      Device_20  Analyze the following table data and provide a...
(43, 2)


In [ ]:
train.dtypes

DeviceMaskedID                object
data (train/ test)            object
date                  datetime64[ns]
cpu_usage                    float64
ram_usage                    float64
diskio_usage                 float64
dtype: object

## Load Model

Model configurations

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.float16,
    bnb_4bit_use_double_quant= True,
llm_int8_enable_fp32_cpu_offload= True)

Model name

In [ ]:
# model_name = 'databricks/dolly-v2-3b'
model_name = 'mistralai/Mistral-7B-Instruct-v0.2'
model_tag = 'mistral'

import token for huggingface

In [ ]:
from google.colab import userdata
huggingface_token = userdata.get('huggingface_read')

Load the base model

In [ ]:
orig_model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True,
        token=huggingface_token)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Load the tokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, token=huggingface_token)

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [ ]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training


# Prepare Data

In [ ]:
def tokenize_function(allEntries):
  return tokenizer(allEntries['Instruction'], truncation=True, max_length=1000)

In [ ]:
train_dataset = Dataset.from_dict(train_df)
validation_dataset = Dataset.from_dict(validation_df)
test_dataset = Dataset.from_dict(test_df)

In [ ]:
train_dataset  = train_dataset.map(tokenize_function, remove_columns=["DeviceMaskedID"])
validation_dataset  = validation_dataset.map(tokenize_function, remove_columns=["DeviceMaskedID"])
test_dataset = test_dataset.map(tokenize_function, remove_columns=["DeviceMaskedID"])

Map:   0%|          | 0/163 [00:00<?, ? examples/s]

Map:   0%|          | 0/9 [00:00<?, ? examples/s]

Map:   0%|          | 0/43 [00:00<?, ? examples/s]

## LORA

Lora config

In [ ]:
peft_type = "lora"
max_length=3800
lr=1e-4
num_epochs=1
batch_size=1
rank = 8
max_steps = 5
logging_steps = 1
evaluation_strategy = "steps"
logging_strategy = "steps"
eval_steps = 10
logging_steps = 10
save_strategy = "steps"
save_steps = 10
do_eval = True
save_total_limit = 10
load_best_model_at_end=True
warmup_steps = 2

In [ ]:
lora_config = LoraConfig(
    r=100,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    # task_type=TaskType.SEQ_2_SEQ_LM # FLAN-T5
    task_type=TaskType.CAUSAL_LM, # decoder to decoder
)

In [ ]:
peft_lora_model = get_peft_model(orig_model,
                            lora_config)
print(peft_lora_model.print_trainable_parameters())
# print(print_number_of_trainable_model_parameters(peft_model))

trainable params: 42,598,400 || all params: 7,284,330,496 || trainable%: 0.5848
None


In [ ]:
peft_model_id = f'{model_tag}_{peft_type}_rank{rank}_ep{num_epochs}_bs{batch_size}_lr{lr}_maxtoken{max_length}_short_prompt'
output_dir = f"/content/drive/MyDrive/Colab Notebooks/checkpoints/{peft_model_id}"

In [ ]:
peft_lora_training_args = TrainingArguments(
    output_dir=output_dir,
    auto_find_batch_size=True,
    learning_rate=lr, # Higher learning rate than full fine-tuning.
    num_train_epochs=num_epochs,
    do_eval=do_eval,
    evaluation_strategy=evaluation_strategy,
    eval_steps=eval_steps,
    logging_strategy=logging_strategy,
    logging_steps=logging_steps,
    save_steps=save_steps,
    # max_steps=5
    save_total_limit=save_total_limit,
    # load_best_model_at_end=load_best_model_at_end,
    warmup_steps=warmup_steps,
)

In [ ]:
data_collator = DataCollatorForLanguageModeling(
  tokenizer,
  mlm=False
)

In [ ]:
validation_dataset

Dataset({
    features: ['Instruction', 'input_ids', 'attention_mask'],
    num_rows: 9
})

Train Lora

In [ ]:
peft_lora_trainer = Trainer(
  model=peft_lora_model,
  args=peft_lora_training_args,
  data_collator=data_collator,
  train_dataset=train_dataset,
  eval_dataset=validation_dataset
)

In [ ]:
peft_lora_train_results = peft_lora_trainer.train()

Step,Training Loss,Validation Loss
10,0.518000,0.452272
20,0.403100,0.382466
30,0.400700,0.355698
40,0.416200,0.351059
50,0.404700,0.354375
60,0.353300,0.351859
70,0.356900,0.351595
80,0.389100,0.350184
90,0.444400,0.350238
100,0.415800,0.349934


/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:611: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-66542527-5f2b50c746c5ed9349cf8421;5e0d9426-6d69-431a-8b5a-d50f0751710c)

Cannot access gated repo for url https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2/resolve/main/config.json.
Access to model mistralai/Mistral-7B-Instruct-v0.2 is restricted. You must be authenticated to access it. - silently ignoring the lookup for the file config.json in mistralai/Mistral-7B-Instruct-v0.2.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:200: UserWarning: Could not find a config file in mistralai/Mistral-7B-Instruct-v0.2 - will assume that the vocabulary was not modified.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:611: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-66542543-1ee989ac749f4ec5738

In [ ]:
peft_lora_trainer.log_metrics("train", peft_lora_train_results.metrics)
peft_lora_trainer.save_metrics("train", peft_lora_train_results.metrics)

***** train metrics *****
  epoch                    =        1.0
  total_flos               =  6021014GF
  train_loss               =      0.396
  train_runtime            = 0:08:09.45
  train_samples_per_second =      0.333
  train_steps_per_second   =      0.333


In [ ]:
lora_val_results = peft_lora_trainer.evaluate()
print(lora_val_results)

{'eval_loss': 0.34801924228668213, 'eval_runtime': 7.8914, 'eval_samples_per_second': 1.14, 'eval_steps_per_second': 0.253, 'epoch': 1.0}


Save trained model

In [ ]:
peft_lora_model_path="./peft-lora-dialogue-summary-checkpoint-local"
peft_lora_model_path = f'/content/drive/MyDrive/Colab Notebooks/checkpoints/{model_tag}_{peft_type}_rank{rank}_ep{num_epochs}_bs{batch_size}_lr{lr}_maxtoken{max_length}_short_prompt'

In [ ]:
peft_lora_trainer.model.save_pretrained(peft_lora_model_path)
tokenizer.save_pretrained(peft_lora_model_path)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:611: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-665426ff-65c7c1793dc8a9dc4bc482f6;73eb352c-f914-4fbb-b9db-157cbe6b088a)

Cannot access gated repo for url https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2/resolve/main/config.json.
Access to model mistralai/Mistral-7B-Instruct-v0.2 is restricted. You must be authenticated to access it. - silently ignoring the lookup for the file config.json in mistralai/Mistral-7B-Instruct-v0.2.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:200: UserWarning: Could not find a config file in mistralai/Mistral-7B-Instruct-v0.2 - will assume that the vocabulary was not modified.
  warnings.warn(


('/content/drive/MyDrive/Colab Notebooks/checkpoints/mistral_lora_rank8_ep1_bs1_lr0.0001_maxtoken3800_short_prompt/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebooks/checkpoints/mistral_lora_rank8_ep1_bs1_lr0.0001_maxtoken3800_short_prompt/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebooks/checkpoints/mistral_lora_rank8_ep1_bs1_lr0.0001_maxtoken3800_short_prompt/tokenizer.model',
 '/content/drive/MyDrive/Colab Notebooks/checkpoints/mistral_lora_rank8_ep1_bs1_lr0.0001_maxtoken3800_short_prompt/added_tokens.json',
 '/content/drive/MyDrive/Colab Notebooks/checkpoints/mistral_lora_rank8_ep1_bs1_lr0.0001_maxtoken3800_short_prompt/tokenizer.json')

Load the pre trained model

In [ ]:
lora_path = "/content/drive/MyDrive/Colab Notebooks/checkpoints/mistral_lora_rank8_ep1_bs1_lr0.0001_maxtoken3800_short_prompt/" ## Use your path

In [ ]:
ft_lora_config = PeftConfig.from_pretrained(lora_path)
# model_ = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2", load_in_4bit=True, device_map="auto")
ft_lora_model = PeftModel.from_pretrained(orig_model, lora_path, device_map="auto")

ft_lora_tokenizer = AutoTokenizer.from_pretrained(lora_path, device_map='auto')

### LORA Test

In [ ]:
def func_to_inference_greedy(model_, tokenizer_, text_):
  inputs = tokenizer_(text_, return_tensors="pt")

  # Greedy Search
  outputs = model_.generate(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], do_sample=False, max_new_tokens=800)
  # outputs = model_.generate(inputs, do_sample=False, max_new_tokens=800)

  pred_ = tokenizer_.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]

  pred_ = pred_.split('Summary:\n')[1]

  return pred_

In [ ]:
# test_df.head(1).apply(lambda a: func_to_inference_greedy(ft_lora_model, ft_lora_tokenizer, a['Instruction']), axis=1)

In [ ]:
test_df['lora_inference'] = test_df.apply(lambda a: func_to_inference_greedy(ft_lora_model, ft_lora_tokenizer, a['Instruction']), axis=1)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1413: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_

LORA Save Test Data

In [ ]:
test_df.to_csv(f"/content/drive/MyDrive/Colab Notebooks/output/peft_lora.csv", index=False)

## LORA Evaluate the Model Quantitatively (with ROUGE Metric)

In [ ]:
test_df.head()

,DeviceMaskedID,Instruction,lora_inference
0,Device_4,Analyze the following table data and provide a...,"1. CPU usage spiked to 36.98% on August 18, ..."
1,Device_8,Analyze the following table data and provide a...,1. CPU usage dropped significantly from 56.8...
2,Device_16,Analyze the following table data and provide a...,1. Disk I/O usage remained consistently high...
3,Device_17,Analyze the following table data and provide a...,1. CPU usage dropped significantly from 49.3...
4,Device_20,Analyze the following table data and provide a...,1. RAM usage consistently remained above 88%...


In [ ]:
test_merged = test_df.merge(labels[["DeviceMaskedID", "insight_2"]], on=["DeviceMaskedID"])
test_labels = test_merged.rename(columns={"insight_2": "Original_Summary"})
test_labels

,DeviceMaskedID,Instruction,lora_inference,Original_Summary
0,Device_4,Analyze the following table data and provide a...,"1. CPU usage spiked to 36.98% on August 18, ...",1. Missing Data: The 'diskio_usage' column has...
1,Device_8,Analyze the following table data and provide a...,1. CPU usage dropped significantly from 56.8...,1. Missing Data: The 'diskio_usage' column has...
2,Device_16,Analyze the following table data and provide a...,1. Disk I/O usage remained consistently high...,"After analyzing the dataset, the top 3 aberrat..."
3,Device_17,Analyze the following table data and provide a...,1. CPU usage dropped significantly from 49.3...,"1. On 223-3-3, there is a significant increase..."
4,Device_20,Analyze the following table data and provide a...,1. RAM usage consistently remained above 88%...,"Based on the provided data, the top 3 aberrati..."
5,Device_25,Analyze the following table data and provide a...,1. Disk I/O usage has been consistently high...,"After analyzing the dataset, the top 3 aberrat..."
6,Device_27,Analyze the following table data and provide a...,1. CPU usage dropped significantly from 65.5...,1. High CPU usage: The highest CPU usage recor...
7,Device_29,Analyze the following table data and provide a...,1. CPU usage dropped significantly from 43.0...,"1. On 223-3-24, there is a significant drop in..."
8,Device_31,Analyze the following table data and provide a...,1. Disk I/O usage has been consistently high...,"Based on the provided data, the top 3 aberrati..."
9,Device_33,Analyze the following table data and provide a...,"1. RAM usage has been consistently high, ran...","Based on the provided data, the top 3 aberrati..."


In [ ]:
test_labels.to_csv(f"/content/drive/MyDrive/Colab Notebooks/output/peft_lora.csv", index=False)

In [ ]:
rouge = evaluate.load('rouge')

In [ ]:
lora_model_results = rouge.compute(
    predictions=test_labels["lora_inference"],
    references=test_labels["Original_Summary"],
    use_aggregator=True,
    use_stemmer=True,
)

In [ ]:
lora_model_results

{'rouge1': 0.30027732630380605,
 'rouge2': 0.07100238972133438,
 'rougeL': 0.205375069979091,
 'rougeLsum': 0.27181104930602495}

## Prompt Tuning

Prompt tuning config

In [ ]:
peft_type = "prompt_tuning"
max_length=3800
lr=1e-4
num_epochs=1
batch_size=1
rank = 8
max_steps = 5
logging_steps = 1
evaluation_strategy = "steps"
logging_strategy = "steps"
eval_steps = 10
logging_steps = 10
save_strategy = "steps"
save_steps = 10
do_eval = True
save_total_limit = 10
load_best_model_at_end=True
warmup_steps = 2

In [ ]:
config = AutoConfig.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
model_hidden_size = config.hidden_size

In [ ]:
prompt_peft_model = prepare_model_for_kbit_training(orig_model)

In [ ]:
prompt_ = "Analyze the following table data and provide a creative summary in not more than 3 points:"

In [ ]:
prompt_peft_config = PromptTuningConfig(
  task_type=TaskType.CAUSAL_LM, # decoder to decoder
  prompt_tuning_init=PromptTuningInit.TEXT,
  num_virtual_tokens=100,
  prompt_tuning_init_text=prompt_,
  base_model_name_or_path=model_name,
  tokenizer_name_or_path=model_name
)

In [ ]:
prompt_peft_model = get_peft_model(prompt_peft_model, prompt_peft_config)
print(prompt_peft_model.print_trainable_parameters())


trainable params: 409,600 || all params: 7,284,740,096 || trainable%: 0.0056
None


In [ ]:
prompt_peft_model_id = f'{model_tag}_{peft_type}_rank{rank}_ep{num_epochs}_bs{batch_size}_lr{lr}_maxtoken{max_length}_short_prompt'
prompt_peft_output_dir = f"/content/drive/MyDrive/Colab Notebooks/checkpoints/{prompt_peft_model_id}"

In [ ]:
prompt_peft_training_args = TrainingArguments(
    output_dir=prompt_peft_output_dir,
    auto_find_batch_size=True,
    learning_rate=lr, # Higher learning rate than full fine-tuning.
    num_train_epochs=num_epochs,
    do_eval=do_eval,
    evaluation_strategy=evaluation_strategy,
    eval_steps=eval_steps,
    logging_strategy=logging_strategy,
    logging_steps=logging_steps,
    save_steps=save_steps,
    # max_steps=5
    save_total_limit=save_total_limit,
    # load_best_model_at_end=load_best_model_at_end,
    warmup_steps=warmup_steps,
)

In [ ]:
data_collator = DataCollatorForLanguageModeling(
  tokenizer,
  mlm=False
)

In [ ]:
# train[""]

In [ ]:
train_dataset

Dataset({
    features: ['Instruction', 'input_ids', 'attention_mask'],
    num_rows: 163
})

Train model using Prompt tuning

In [ ]:
prompt_peft_trainer = Trainer(
  model=prompt_peft_model,
  args=prompt_peft_training_args,
  data_collator=data_collator,
  train_dataset=train_dataset,
  eval_dataset=validation_dataset
)

In [ ]:
prompt_peft_train_results = prompt_peft_trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss
10,0.431400,0.370172
20,0.381000,0.366946
30,0.395600,0.363781
40,0.386600,0.362395


/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:611: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-66542e64-1d348ddc32f0bc9746878fe8;cfac98cb-f869-4300-8e99-7d1cd1a5ad14)

Cannot access gated repo for url https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2/resolve/main/config.json.
Access to model mistralai/Mistral-7B-Instruct-v0.2 is restricted. You must be authenticated to access it. - silently ignoring the lookup for the file config.json in mistralai/Mistral-7B-Instruct-v0.2.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:200: UserWarning: Could not find a config file in mistralai/Mistral-7B-Instruct-v0.2 - will assume that the vocabulary was not modified.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an ex

In [ ]:
prompt_peft_trainer.log_metrics("train", prompt_peft_train_results.metrics)
prompt_peft_trainer.save_metrics("train", prompt_peft_train_results.metrics)

***** train metrics *****
  epoch                    =        1.0
  total_flos               =  6481531GF
  train_loss               =     0.3992
  train_runtime            = 0:13:10.47
  train_samples_per_second =      0.206
  train_steps_per_second   =      0.052


In [ ]:
val_results = prompt_peft_trainer.evaluate()
print(val_results)

{'eval_loss': 0.36236536502838135, 'eval_runtime': 11.8523, 'eval_samples_per_second': 0.759, 'eval_steps_per_second': 0.169, 'epoch': 1.0}


In [ ]:
prompt_peft_model_path="./peft-dialogue-summary-checkpoint-local"
prompt_peft_model_path = f'/content/drive/MyDrive/Colab Notebooks/checkpoints/{model_tag}_{peft_type}_rank{rank}_ep{num_epochs}_bs{batch_size}_lr{lr}_maxtoken{max_length}_short_prompt'

In [ ]:
prompt_peft_trainer

In [ ]:
# peft_trainer.model.save_pretrained(save_directory=f"{peft_model_path}_saikat")

Save prefix tuned model

In [ ]:
prompt_peft_trainer.model.save_pretrained(prompt_peft_model_path)
tokenizer.save_pretrained(prompt_peft_model_path)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:611: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-66543143-2e0f3af1482c049f4df1811b;ccfd1d13-a302-4bb9-9bf2-d056330b9a62)

Cannot access gated repo for url https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2/resolve/main/config.json.
Access to model mistralai/Mistral-7B-Instruct-v0.2 is restricted. You must be authenticated to access it. - silently ignoring the lookup for the file config.json in mistralai/Mistral-7B-Instruct-v0.2.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:200: UserWarning: Could not find a config file in mistralai/Mistral-7B-Instruct-v0.2 - will assume that the vocabulary was not modified.
  warnings.warn(


('/content/drive/MyDrive/Colab Notebooks/checkpoints/mistral_prompt_tuning_rank8_ep1_bs1_lr0.0001_maxtoken3800_short_prompt/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebooks/checkpoints/mistral_prompt_tuning_rank8_ep1_bs1_lr0.0001_maxtoken3800_short_prompt/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebooks/checkpoints/mistral_prompt_tuning_rank8_ep1_bs1_lr0.0001_maxtoken3800_short_prompt/tokenizer.model',
 '/content/drive/MyDrive/Colab Notebooks/checkpoints/mistral_prompt_tuning_rank8_ep1_bs1_lr0.0001_maxtoken3800_short_prompt/added_tokens.json',
 '/content/drive/MyDrive/Colab Notebooks/checkpoints/mistral_prompt_tuning_rank8_ep1_bs1_lr0.0001_maxtoken3800_short_prompt/tokenizer.json')

### Load saved Prompt tuned model and Test it

In [ ]:
prompt_tuned_model_path = "/content/drive/MyDrive/Colab Notebooks/checkpoints/mistral_prompt_tuning_rank8_ep1_bs1_lr0.0001_maxtoken3800_short_prompt/" ## Use your path

In [ ]:
prompt_tuned_config = PeftConfig.from_pretrained(prompt_tuned_model_path)

In [ ]:
prompt_tuned_config

PromptTuningConfig(peft_type=<PeftType.PROMPT_TUNING: 'PROMPT_TUNING'>, auto_mapping=None, base_model_name_or_path='mistralai/Mistral-7B-Instruct-v0.2', revision=None, task_type='CAUSAL_LM', inference_mode=True, num_virtual_tokens=100, token_dim=4096, num_transformer_submodules=1, num_attention_heads=32, num_layers=32, prompt_tuning_init='TEXT', prompt_tuning_init_text='Analyze the following table data and provide a creative summary in not more than 3 points:', tokenizer_name_or_path='mistralai/Mistral-7B-Instruct-v0.2', tokenizer_kwargs=None)

In [ ]:
# config = PeftConfig.from_pretrained(path)
ft_prompt_peft_model = PeftModel.from_pretrained(orig_model, prompt_tuned_model_path, device_map="auto")

ft_prompt_peft_tokenizer = AutoTokenizer.from_pretrained(prompt_tuned_model_path, device_map='auto')

In [ ]:
def func_to_inference_greedy(model_, tokenizer_, text_):
  inputs = tokenizer_(text_, return_tensors="pt")

  # Greedy Search
  outputs = model_.generate(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], do_sample=False, max_new_tokens=800)

  pred_ = tokenizer_.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]

  pred_ = pred_.split('Summary:\n')[1]

  return pred_

In [ ]:
test_df.head(1).apply(lambda a: func_to_inference_greedy(ft_prompt_peft_model, ft_prompt_peft_tokenizer, a['Instruction']), axis=1)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1413: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/peft_model.py:1579: UserWarning: Position ids are not supported for parameter efficient tuning. Ignoring position ids.
  warnings.warn("Position ids are not supported for parameter efficient tuning. Ignoring position ids.")


0      1. CPU usage has been fluctuating between 4-...
dtype: object

In [ ]:
test_df['prompt_inference'] = test_df.apply(lambda a: func_to_inference_greedy(ft_prompt_peft_model, ft_prompt_peft_tokenizer, a['Instruction']), axis=1)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for o

In [ ]:
test_df

,DeviceMaskedID,Instruction,lora_inference,prompt_inference
0,Device_4,Analyze the following table data and provide a...,"1. CPU usage spiked to 36.98% on August 18, ...",1. CPU usage has been fluctuating between 4-...
1,Device_8,Analyze the following table data and provide a...,1. CPU usage dropped significantly from 56.8...,1. CPU usage dropped significantly from 56.8...
2,Device_16,Analyze the following table data and provide a...,1. Disk I/O usage remained consistently high...,1. Disk I/O usage has been consistently high...
3,Device_17,Analyze the following table data and provide a...,1. CPU usage dropped significantly from 49.3...,"1. CPU usage has been consistently high, ave..."
4,Device_20,Analyze the following table data and provide a...,1. RAM usage consistently remained above 88%...,"1. The RAM usage has been consistently high,..."
5,Device_25,Analyze the following table data and provide a...,1. Disk I/O usage has been consistently high...,1. Disk I/O usage has been consistently high...
6,Device_27,Analyze the following table data and provide a...,1. CPU usage dropped significantly from 65.5...,"1. CPU usage has been consistently high, ave..."
7,Device_29,Analyze the following table data and provide a...,1. CPU usage dropped significantly from 43.0...,"1. CPU usage has been consistently high, ave..."
8,Device_31,Analyze the following table data and provide a...,1. Disk I/O usage has been consistently high...,1. Disk I/O usage has been consistently high...
9,Device_33,Analyze the following table data and provide a...,"1. RAM usage has been consistently high, ran...","1. RAM usage has been consistently high, ave..."


## Prompt Evaluate the Model Quantitatively (with ROUGE Metric)

In [ ]:
test_merged = test_df.merge(labels[["DeviceMaskedID", "insight_2"]], on=["DeviceMaskedID"])
prompt_tuning_test_labels = test_merged.rename(columns={"insight_2": "Original_Summary"})
prompt_tuning_test_labels

,DeviceMaskedID,Instruction,lora_inference,prompt_inference,Original_Summary
0,Device_4,Analyze the following table data and provide a...,"1. CPU usage spiked to 36.98% on August 18, ...",1. CPU usage has been fluctuating between 4-...,1. Missing Data: The 'diskio_usage' column has...
1,Device_8,Analyze the following table data and provide a...,1. CPU usage dropped significantly from 56.8...,1. CPU usage dropped significantly from 56.8...,1. Missing Data: The 'diskio_usage' column has...
2,Device_16,Analyze the following table data and provide a...,1. Disk I/O usage remained consistently high...,1. Disk I/O usage has been consistently high...,"After analyzing the dataset, the top 3 aberrat..."
3,Device_17,Analyze the following table data and provide a...,1. CPU usage dropped significantly from 49.3...,"1. CPU usage has been consistently high, ave...","1. On 223-3-3, there is a significant increase..."
4,Device_20,Analyze the following table data and provide a...,1. RAM usage consistently remained above 88%...,"1. The RAM usage has been consistently high,...","Based on the provided data, the top 3 aberrati..."
5,Device_25,Analyze the following table data and provide a...,1. Disk I/O usage has been consistently high...,1. Disk I/O usage has been consistently high...,"After analyzing the dataset, the top 3 aberrat..."
6,Device_27,Analyze the following table data and provide a...,1. CPU usage dropped significantly from 65.5...,"1. CPU usage has been consistently high, ave...",1. High CPU usage: The highest CPU usage recor...
7,Device_29,Analyze the following table data and provide a...,1. CPU usage dropped significantly from 43.0...,"1. CPU usage has been consistently high, ave...","1. On 223-3-24, there is a significant drop in..."
8,Device_31,Analyze the following table data and provide a...,1. Disk I/O usage has been consistently high...,1. Disk I/O usage has been consistently high...,"Based on the provided data, the top 3 aberrati..."
9,Device_33,Analyze the following table data and provide a...,"1. RAM usage has been consistently high, ran...","1. RAM usage has been consistently high, ave...","Based on the provided data, the top 3 aberrati..."


In [ ]:
prompt_tuning_test_labels.to_csv(f"/content/drive/MyDrive/Colab Notebooks/output/prompt_tuning.csv", index=False)

In [ ]:
rouge = evaluate.load('rouge')

In [ ]:
prompt_model_results = rouge.compute(
    predictions=prompt_tuning_test_labels["prompt_inference"],
    references=prompt_tuning_test_labels["Original_Summary"],
    use_aggregator=True,
    use_stemmer=True,
)

In [ ]:
print(prompt_model_results)

{'rouge1': 0.24554145201188682, 'rouge2': 0.04416392143090263, 'rougeL': 0.16966888046466497, 'rougeLsum': 0.2249266376871718}
